<a href="https://colab.research.google.com/github/Firojpaudel/Reasoning_Model_/blob/main/Creating_reasoning_model_GRPO.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!rm -rf /content/sample_data

#### Before Diving In:

**What is GRPO?**
- Stands for Group Relative Policy Optimization. Its is a reinforcement learning technique that helps models develop reasoning capabilities autonomously.
- First introduced the concept on DeepSeek paper.

---

#### **Why Unsloth?**:
- Well, thanks to Unsloth, training a larger params model ( $15B$ or so) on customer-grade GPUs like that of Colab and Kaggle is possible
  - Reason: Due to some major optimizations done on the **transformers** library.
---
> Major Props to Daniel and team 🙌!

In [2]:
## Installation
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    !pip install --no-deps unsloth vllm==0.8.5.post1

In [3]:
#@title Colab Extra Install { display-mode: "form" }
%%capture
import os
if "COLAB_" not in "".join(os.environ.keys()):
    !pip install unsloth vllm
else:
    !pip install --no-deps unsloth vllm==0.8.5.post1
    # [NOTE] Do the below ONLY in Colab! Use [[pip install unsloth vllm]]
    # Skip restarting message in Colab
    import sys, re, requests; modules = list(sys.modules.keys())
    for x in modules: sys.modules.pop(x) if "PIL" in x or "google" in x else None
    !pip install --no-deps bitsandbytes accelerate xformers==0.0.29.post3 peft "trl==0.15.2" triton cut_cross_entropy unsloth_zoo
    !pip install sentencepiece protobuf "datasets>=3.4.1" huggingface_hub hf_transfer
    !pip install transformers==4.51.3

    # vLLM requirements - vLLM breaks Colab due to reinstalling numpy
    f = requests.get("https://raw.githubusercontent.com/vllm-project/vllm/refs/heads/main/requirements/common.txt").content
    with open("vllm_requirements.txt", "wb") as file:
        file.write(re.sub(rb"(transformers|numpy|xformers)[^\n]{1,}\n", b"", f))
    !pip install -r vllm_requirements.txt

#### The Main Process:


In [4]:
!pip install langid -qq

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.9/1.9 MB 21.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done


In [5]:
from unsloth import FastLanguageModel
import torch
max_seq_length = 1024 # Can increase for longer reasoning traces
lora_rank = 32 # Larger rank = smarter, but slower

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/DeepSeek-R1-0528-Qwen3-8B",
    max_seq_length = max_seq_length,
    load_in_4bit = True, # False for LoRA 16bit
    fast_inference = True, # Enable vLLM fast inference
    max_lora_rank = lora_rank,
    gpu_memory_utilization = 0.7, # Reduce if out of memory
)

model = FastLanguageModel.get_peft_model(
    model,
    r = lora_rank, # Choose any number > 0 ! Suggested 8, 16, 32, 64, 128
    target_modules = [
        "q_proj", "k_proj", "v_proj", "o_proj",
        "gate_proj", "up_proj", "down_proj",
    ],
    lora_alpha = lora_rank*2, # *2 speeds up training
    use_gradient_checkpointing = "unsloth", # Reduces memory usage
    random_state = 3407,
)

🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
INFO 06-09 13:30:09 [importing.py:53] Triton module has been replaced with a placeholder.
INFO 06-09 13:30:09 [__init__.py:239] Automatically detected platform cuda.


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


==((====))==  Unsloth 2025.6.1: Fast Qwen3 patching. Transformers: 4.51.3. vLLM: 0.8.5.post1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.6.0+cu124. CUDA: 7.5. CUDA Toolkit: 12.4. Triton: 3.2.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.29.post3. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


Unrecognized keys in `rope_scaling` for 'rope_type'='yarn': {'attn_factor'}


Unsloth: vLLM loading unsloth/deepseek-r1-0528-qwen3-8b-unsloth-bnb-4bit with actual GPU utilization = 69.34%
Unsloth: Your GPU has CUDA compute capability 7.5 with VRAM = 14.74 GB.
Unsloth: Using conservativeness = 1.0. Chunked prefill tokens = 2048. Num Sequences = 160.
Unsloth: vLLM's KV Cache can use up to 3.7 GB. Also swap space = 0 GB.


...

tokenizer_config.json:   0%|          | 0.00/11.0k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/11.4M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/472 [00:00<?, ?B/s]

chat_template.jinja:   0%|          | 0.00/5.26k [00:00<?, ?B/s]

Unsloth 2025.6.1 patched 36 layers with 36 QKV layers, 36 O layers and 36 MLP layers.


#### Optimizing for GRPO Chat Template


In [6]:
reasoning_start = None
reasoning_end = None
user_token = None
assistant_token = None

for token in tokenizer.get_added_vocab().keys():
    if "think" in token and "/" in token:
        reasoning_end = token
    elif "think" in token:
        reasoning_start = token
    elif "user" in token:
        user_token = token
    elif "assistant" in token:
        assistant_token = token

system_prompt = \
f"""You are given a problem.
Think about the problem and provide your working out.
You must think in English."""
system_prompt

'You are given a problem.\nThink about the problem and provide your working out.\nYou must think in English.'

In [7]:
print(tokenizer.apply_chat_template([
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"<think>I think it's 2.2</think>2"},
    {"role" : "user", "content" : "What is 1+1?"},
    {"role" : "assistant", "content" : f"<think>I think it's 2.2</think>2"},
], tokenize = False, add_generation_prompt = True))

<｜begin▁of▁sentence｜><｜User｜>What is 1+1?<｜Assistant｜>2<｜end▁of▁sentence｜><｜User｜>What is 1+1?<｜Assistant｜>2<｜end▁of▁sentence｜><｜Assistant｜>


#### Data prep:


In [8]:
from datasets import load_dataset, concatenate_datasets, Dataset

# Function to format datasets for GRPO
def format_example(example, dataset_name):
    if dataset_name == "open-r1/DAPO-Math-17k-Processed":
        prompt = example.get("question", "")
        reasoning = example.get("reasoning", "Reasoning: Break down the math problem step by step to find the solution.")
        answer = example.get("answer", "")
    elif dataset_name == "cais/mmlu":
        prompt = example.get("question", "")
        reasoning = "Reasoning: Analyze the question, consider all options, and deduce the correct solution."
        answer = example.get("answer", "")
    elif dataset_name == "glue":  # MultiNLI
        prompt = f"Premise: {example['premise']} Hypothesis: {example['hypothesis']}"
        reasoning = "Reasoning: Determine if the hypothesis entails, contradicts, or is neutral to the premise."
        answer = str(example["label"])  # 0: entailment, 1: neutral, 2: contradiction
    elif dataset_name == "xnli":
        # XNLI structure: premise is a dict, hypothesis is a list of translations
        # Check if 'hypothesis' is a list and not empty before accessing index 0
        hypothesis_en = example.get('hypothesis')
        if isinstance(hypothesis_en, list) and len(hypothesis_en) > 0:
            prompt = f"Premise: {example['premise']['en']} Hypothesis: {hypothesis_en[0]}"  # Use first hypothesis translation (English)
        else:
            # Handle cases where hypothesis is not a list or is empty
            prompt = f"Premise: {example['premise'].get('en', '')} Hypothesis: " # Provide default or skip example

        reasoning = "Reasoning: Evaluate the English premise and hypothesis for entailment, contradiction, or neutrality."
        answer = str(example.get("label", -1))
    elif dataset_name == "truthful_qa":
        prompt = example["question"]
        reasoning = "Reasoning: Avoid misconceptions, critically assess the question, and provide a truthful path."
        answer = example["best_answer"]
    elif dataset_name == "lukaemon/bbh":
        prompt = example.get("input", "")
        reasoning = "Reasoning: Tackle this complex task step by step, considering all angles."
        answer = example.get("target", "")
    elif dataset_name == "opus100":
        prompt = example["translation"]["en"]  # English prompt
        reasoning = "Reasoning: Understand the English input, translate and respond naturally in Nepali."
        answer = example["translation"]["ne"]
    # elif dataset_name == "facebook/flores-200":
    #     prompt = example["sentence_eng_Latn"]
    #     reasoning = "Reasoning: Comprehend the English sentence, respond naturally in Nepali."
    #     answer = example["sentence_nep_Nepali"]
    else:  # Synthetic Nepali
        prompt = example["prompt"]
        reasoning = example["reasoning"]
        answer = example["answer"]
    return {
        "prompt": prompt,
        "chosen": f"<reasoning>{reasoning}</reasoning><answer>{answer}</answer>",
        "rejected": "<reasoning>Incomplete or incorrect reasoning.</reasoning><answer>Wrong or irrelevant response.</answer>"
    }

In [9]:
# Load datasets
dapo_math = load_dataset("open-r1/DAPO-Math-17k-Processed", "en", split="train")
mmlu = load_dataset("cais/mmlu", "all", split="auxiliary_train")
multinli = load_dataset("glue", "mnli", split="train[:10000]")  # Subset for speed
xnli = load_dataset("xnli", "all_languages", split="train[:10000]")  # Subset for speed
truthfulqa = load_dataset("truthful_qa", "generation", split="validation")
bbh_navigate = load_dataset("lukaemon/bbh", "navigate", split="test")  # One task
bbh_boolean = load_dataset("lukaemon/bbh", "boolean_expressions", split="test")  # Another task
opus = load_dataset("opus100", "en-ne", split="train[:5000]")  # Subset for speed
# flores = load_dataset("facebook/flores-200", split="dev")  # Nepali included

README.md:   0%|          | 0.00/3.44k [00:00<?, ?B/s]

en/train-00000-of-00001.parquet:   0%|          | 0.00/5.23M [00:00<?, ?B/s]

Generating train split:   0%|          | 0/14116 [00:00<?, ? examples/s]

README.md:   0%|          | 0.00/53.2k [00:00<?, ?B/s]

dataset_infos.json:   0%|          | 0.00/138k [00:00<?, ?B/s]

...

train-00000-of-00001.parquet:   0%|          | 0.00/23.9M [00:00<?, ?B/s]

validation-00000-of-00001.parquet:   0%|          | 0.00/101k [00:00<?, ?B/s]

Generating test split:   0%|          | 0/2000 [00:00<?, ? examples/s]

Generating train split:   0%|          | 0/406381 [00:00<?, ? examples/s]

Generating validation split:   0%|          | 0/2000 [00:00<?, ? examples/s]

In [10]:
# Formatting the datasets and removing original columns
dapo_math = dapo_math.map(lambda x: format_example(x, "open-r1/DAPO-Math-17k-Processed"), remove_columns=dapo_math.column_names)
mmlu = mmlu.map(lambda x: format_example(x, "cais/mmlu"), remove_columns=mmlu.column_names)
multinli = multinli.map(lambda x: format_example(x, "glue"), remove_columns=multinli.column_names)
xnli = xnli.map(lambda x: format_example(x, "xnli"), remove_columns=xnli.column_names)
truthfulqa = truthfulqa.map(lambda x: format_example(x, "truthful_qa"), remove_columns=truthfulqa.column_names)
bbh_navigate = bbh_navigate.map(lambda x: format_example(x, "lukaemon/bbh"), remove_columns=bbh_navigate.column_names)
bbh_boolean = bbh_boolean.map(lambda x: format_example(x, "lukaemon/bbh"), remove_columns=bbh_boolean.column_names)
opus = opus.filter(lambda x: x["translation"]["en"] and x["translation"]["ne"]).map(lambda x: format_example(x, "opus100"), remove_columns=opus.column_names)

Map:   0%|          | 0/14116 [00:00<?, ? examples/s]

Map:   0%|          | 0/99842 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/10000 [00:00<?, ? examples/s]

Map:   0%|          | 0/817 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Map:   0%|          | 0/250 [00:00<?, ? examples/s]

Filter:   0%|          | 0/5000 [00:00<?, ? examples/s]

Map:   0%|          | 0/5000 [00:00<?, ? examples/s]

In [11]:
# Combine and shuffle
combined_dataset = concatenate_datasets([dapo_math, mmlu, multinli, xnli, truthfulqa, bbh_navigate, bbh_boolean, opus])
combined_dataset = combined_dataset.shuffle(seed=3407)

In [12]:
#Inspecting ...
combined_dataset

Dataset({
    features: ['prompt', 'chosen', 'rejected'],
    num_rows: 140275
})

In [13]:
combined_dataset[5000]

{'prompt': "An application wants access to the keyring '%s', but it is locked",
 'chosen': "<reasoning>Reasoning: Understand the English input, translate and respond naturally in Nepali.</reasoning><answer>एउटा अज्ञात अनुप्रयोगले '%s' किरिङमा पहुँच प्राप्त गर्न चाहेको छ, तर यसलाई ताल्चा लगाइएको छ</answer>",
 'rejected': '<reasoning>Incomplete or incorrect reasoning.</reasoning><answer>Wrong or irrelevant response.</answer>'}

#### Creating a regex format to match the reasoning selections and answers:


In [14]:
import re

# Add optional EOS token matching
solution_end_regex = rf"{reasoning_end}(.*)"

match_format = re.compile(solution_end_regex, re.DOTALL)
match_format

re.compile(r'</think>(.*)', re.DOTALL|re.UNICODE)

In [15]:
match_format.findall(
    "Let me think!</think>"\
    f"Hence, the solution is 2.",
)

['Hence, the solution is 2.']

In [16]:
match_format.findall(
    "<think>Let me think!</think>"\
    f"\n\nHence, the solution is 2",
)

['\n\nHence, the solution is 2']

In [17]:
## The reward function creation that matches the format exactly
def match_format_exactly(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0] if isinstance(completion, list) else completion
        if match_format.search(response) is not None:
            score += 3.0
        scores.append(score)
    return scores

In [18]:
##If above one fails: we reward the model if it atleast follows the format partially, by counting each symbol
def match_format_approximately(completions, **kwargs):
    scores = []
    for completion in completions:
        score = 0
        response = completion[0] if isinstance(completion, list) else completion
        score += 0.5 if response.count(reasoning_start) == 1 else -1.0
        score += 0.5 if response.count(reasoning_end) == 1 else -1.0
        scores.append(score)
    return scores

In [19]:
## To extract the generated answers and reward or penalize it!
def check_answer(prompts, completions, **kwargs):
    question = prompts[0][-1]  # Use string directly
    responses = [completion[0] for completion in completions]
    # Extract true answer from 'chosen' field in kwargs
    chosen = kwargs.get('chosen', [None] * len(completions))
    true_answers = [
        re.search(r'<answer>(.*?)</answer>', c).group(1) if c and re.search(r'<answer>(.*?)</answer>', c) else ''
        for c in chosen
    ]
    extracted_responses = [
        guess.group(1) if (guess := match_format.search(r)) is not None else None
        for r in responses
    ]
    scores = []
    for guess, true_answer in zip(extracted_responses, true_answers):
        score = 0
        if guess is None or not true_answer:
            scores.append(-2.0)
            continue
        if guess == true_answer:
            score += 5.0
        elif guess.strip() == true_answer.strip():
            score += 3.5
        else:
            try:
                ratio = float(guess) / float(true_answer)
                if 0.9 <= ratio <= 1.1:
                    score += 2.0
                elif 0.8 <= ratio <= 1.2:
                    score += 1.5
                else:
                    score -= 2.5
            except:
                score -= 4.5
        scores.append(score)
    return scores

In [20]:
match_numbers = re.compile(
    r".*?[\s]{0,}([-]?[\d\.\,]{1,})",
    flags = re.MULTILINE | re.DOTALL
)
print(match_numbers.findall("  0.34  "))
print(match_numbers.findall("  123,456  "))
print(match_numbers.findall("  -0.234  "))
print(match_numbers.findall("17"))

['0.34']
['123,456']
['-0.234']
['17']


In [21]:
import langid

def get_lang(text: str) -> str:
    if not text:
        return "und"
    lang, _ = langid.classify(text)
    return lang


print(get_lang("Hello, How are you")) # This should return en
print(get_lang("Aku berpikir kalau aku adalah kamu")) # This should return id
print(get_lang("我在这里")) # This should return zh
print(get_lang("नमस्ते! कस्तो छ हजुर?")) # This should return ne

en
id
zh
ne


In [22]:
import re

def format_and_language_reward_func(completions, **kwargs):
    scores = []
    for completion in completions:
        # Extract the 'content' string from the dictionary
        response_content = completion[0].get("content", "") if isinstance(completion, list) and len(completion) > 0 else ""

        lang = get_lang(response_content)
        if lang == 'id':
            score = 5.0
        elif lang == 'en':
            score = -3.0
        elif lang == 'zh':
            score = -3.0
        else:
            score = -5.0
        scores.append(score)
    return scores

In [23]:
prompts = [
    [{"role": "assistant", "content": "What is the result of (1 + 2) * 4?"}],
    [{"role": "assistant", "content": "What is the result of (3 + 1) * 2?"}],
]
completions = [
    [{"role": "assistant", "content": "<think>The sum of 1 and 2 is 3, which we multiply by 4 to get 12.</think><answer>(1 + 2) * 4 = 12</answer>"}],
    [{"role": "assistant", "content": "The sum of 3 and 1 is 4, which we multiply by 2 to get 8. So (3 + 1) * 2 = 8."}],
]
format_and_language_reward_func(prompts=prompts, completions=completions)

[-3.0, -3.0]

In [26]:
## Prepping our main function that prints out the generated responses and the true answer, along with another reward function which converts text to float via float and sees if it's the same.
global PRINTED_TIMES
PRINTED_TIMES = 0
global PRINT_EVERY_STEPS
PRINT_EVERY_STEPS = 5

def check_numbers(prompts, completions, **kwargs):
    question = prompts[0][-1]  # Use string directly
    responses = [completion[0] for completion in completions]
    # Extract true answer from 'chosen' field in kwargs
    chosen = kwargs.get('chosen', [None] * len(completions))
    true_answers = [
        re.search(r'<answer>(.*?)</answer>', c).group(1) if c and re.search(r'<answer>(.*?)</answer>', c) else ''
        for c in chosen
    ]
    extracted_responses = [
        guess.group(1) if (guess := match_numbers.search(r)) is not None else None
        for r in responses
    ]
    scores = []
    global PRINTED_TIMES
    global PRINT_EVERY_STEPS
    if PRINTED_TIMES % PRINT_EVERY_STEPS == 0:
        print(
            '*'*20 + f"Question:\n{question}", f"\nAnswer:\n{true_answers[0]}",
            f"\nResponse:\n{responses[0]}", f"\nExtracted:\n{extracted_responses[0]}"
        )
    PRINTED_TIMES += 1
    for guess, true_answer in zip(extracted_responses, true_answers):
        if guess is None or not true_answer:
            scores.append(-2.5)
            continue
        try:
            true_answer = float(true_answer.strip())
            guess = float(guess.strip().replace(",", ""))
            scores.append(3.5 if guess == true_answer else -1.5)
        except:
            scores.append(0)
        continue
    return scores

In [24]:
## Get top 90% prompt length so we don't accidentally truncate them!
def tokenize_prompt(example):
    # Format prompt as a chat message
    chat_format = [{"role": "user", "content": example["prompt"]}]
    tokens = tokenizer.apply_chat_template(chat_format, add_generation_prompt=True, tokenize=True)
    return {"tokens": tokens}

In [25]:
def tokenize_prompt(example):
    # Format prompt as a chat message
    chat_format = [{"role": "user", "content": example["prompt"]}]
    tokens = tokenizer.apply_chat_template(chat_format, add_generation_prompt=True, tokenize=True)
    return {"tokens": tokens}

In [27]:
# Apply the tokenize_prompt function to the dataset to create the 'tokens' column
tokenized = combined_dataset.map(tokenize_prompt)

# Decode first example for inspection
print(tokenizer.decode(tokenized[0]["tokens"]))

# Compute length of tokenized prompts
# Now 'tokens' is a list of token IDs for each example
tokenized = tokenized.map(lambda x: {"L": len(x["tokens"])})

# Find 90th percentile length
import numpy as np # Import numpy if not already imported
maximum_length = int(np.quantile(tokenized["L"], 0.9))
print("Max Length = ", maximum_length)

# Filter samples smaller than 90% max length
dataset = combined_dataset.select(np.where(np.array(tokenized["L"]) <= maximum_length)[0])
del tokenized

Map:   0%|          | 0/140275 [00:00<?, ? examples/s]

<｜begin▁of▁sentence｜><｜User｜>What is the best way for a marine scientist to determine the eating habits of sharks?<｜Assistant｜>


Map:   0%|          | 0/140275 [00:00<?, ? examples/s]

Max Length =  451


### Training the Model

In [28]:
## Now we set up the GRPO Trainer and all configs
max_prompt_length = maximum_length + 1 # + 1 just in case!
max_completion_length = max_seq_length - max_prompt_length

from vllm import SamplingParams
vllm_sampling_params = SamplingParams(
    min_p = 0.1,
    top_p = 1.0,
    top_k = -1,
    seed = 3407,
    stop = [tokenizer.eos_token],
    include_stop_str_in_output = True,
)

from trl import GRPOConfig, GRPOTrainer
training_args = GRPOConfig(
    vllm_sampling_params = vllm_sampling_params,
    temperature = 1.0,
    learning_rate = 5e-6,
    weight_decay = 0.01,
    warmup_ratio = 0.1,
    lr_scheduler_type = "linear",
    optim = "adamw_8bit",
    logging_steps = 1,
    per_device_train_batch_size = 1,
    gradient_accumulation_steps = 1, # Increase to 4 for smoother training
    num_generations = 4, # Decrease if out of memory
    max_prompt_length = max_prompt_length,
    max_completion_length = max_completion_length,
    # num_train_epochs = 1, # Set to 1 for a full training run
    max_steps = 100,
    save_steps = 100,
    report_to = "none", # Can use Weights & Biases
    output_dir = "outputs",

    # For optional training + evaluation
    # fp16_full_eval = True,
    # per_device_eval_batch_size = 4,
    # eval_accumulation_steps = 1,
    # eval_strategy = "steps",
    # eval_steps = 1,
)

Unsloth: We now expect `per_device_train_batch_size` to be a multiple of `num_generations`.
We will change the batch size of 1 to the `num_generations` of 4


In [29]:
# For optional training + evaluation
# new_dataset = dataset.train_test_split(test_size = 0.01)

trainer = GRPOTrainer(
    model = model,
    processing_class = tokenizer,
    reward_funcs = [
        match_format_exactly,
        match_format_approximately,
        check_answer,
        check_numbers,
        format_and_language_reward_func,
    ],
    args = training_args,
    train_dataset = dataset,

    # For optional training + evaluation
    # train_dataset = new_dataset["train"],
    # eval_dataset = new_dataset["test"],
)
trainer.train()

==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 126,439 | Num Epochs = 1 | Total steps = 100
O^O/ \_/ \    Batch size per device = 4 | Gradient accumulation steps = 1
\        /    Data Parallel GPUs = 1 | Total batch size (4 x 1 x 1) = 4
 "-____-"     Trainable parameters = 87,293,952/8,000,000,000 (1.09% trained)


********************Question:
. 
Answer:
1 
Response:
  
Extracted:
None


OutOfMemoryError: CUDA out of memory. Tried to allocate 928.00 MiB. GPU 0 has a total capacity of 14.74 GiB of which 320.12 MiB is free. Process 12661 has 14.43 GiB memory in use. Of the allocated memory 13.76 GiB is allocated by PyTorch, with 128.00 MiB allocated in private pools (e.g., CUDA Graphs), and 51.91 MiB is reserved by PyTorch but unallocated. If reserved but unallocated memory is large try setting PYTORCH_CUDA_ALLOC_CONF=expandable_segments:True to avoid fragmentation.  See documentation for Memory Management  (https://pytorch.org/docs/stable/notes/cuda.html#environment-variables)